In [ ]:
!pip install confluent_kafka
!pip install faker

In [0]:
from confluent_kafka import Producer
import json, random, time
from faker import Faker

# COMMAND ----------

conf = {
    'bootstrap.servers': 'pkc-w77k7w.centralus.azure.confluent.cloud:9092',
    'security.protocol': 'SASL_SSL',  # Fixed typo here
    'sasl.mechanism': 'PLAIN',
    'sasl.username': '36QWHWHZ6CMQYPKH',
    'sasl.password': 'Your Key'  # Ensure this is your actual API secret
}


producer = Producer(conf)
fake = Faker()


def generate_transaction():
    transaction = {
        "transaction_id" : fake.uuid4(),
        "timestamp":int(time.time()),
        "user_id":random.randint(10000,99999),
        "amount":round(random.uniform(5,5000),2),
        "transaction_type":random.choice(["purchase","transfer", "withdrawal"]),
        "location":fake.city(),
        "location_id":random.randint(10000,99999),
        "merchant":fake.company(),
        "card_number":fake.credit_card_number()
    }
    return transaction

def delivery_report(err,msg):
    if err is not None:
        print(f"message is not delivered:{err}")
    else:
        print(f"msg delivered to {msg.topic()}, [{msg.partition()}]")

while True:
    txn = generate_transaction()
    producer.produce("transaction_predict", value = json.dumps(txn), key = str(txn['user_id']), callback = delivery_report)
    producer.flush()
    time.sleep(1)
